In [35]:
%run 2gis_functions.ipynb

Ready: 2023-03-25 07:38:21


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

# Москва запросы и телефоны

In [5]:
rubrics = pd.read_excel('files/final_svod_master.xlsx')[['Рубрика']].iloc[:70]

df_all = dd.read_parquet('files/df_all')
df_filtered = df_all.merge(rubrics, left_on='rubricName', right_on='Рубрика')

df_filtered.to_parquet('df_filtered')

df_filtered = dd.read_parquet('df_filtered')

In [5]:
df = spark.read.parquet("df_filtered")#.withColumn('new_tx',normalize_word(coalesce(col('tx'), lit(''))))

df.createOrReplaceTempView('df')

grouped_df = spark.sql('''

select
    rubricName
    ,year
    ,tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year(date) year
        ,lower(tx) tx
        ,count(*) cnt
    from df
    group by 
        rubricName
        ,year(date)
        ,lower(tx)
        ) t
''')#.where('rn < 16')

In [6]:
%%time
grouped_df.write.mode('overwrite').parquet('df_tx_grouped')

CPU times: total: 78.1 ms
Wall time: 3min 34s


In [41]:
%%time
spark.read.parquet("df_tx_grouped").where('tx is not null and year in (2020,2021,2022,2023)').write.mode('overwrite').parquet('df_tx_grouped_filtered')

CPU times: total: 0 ns
Wall time: 4 s


# Расчет

In [104]:
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def normalize_word(word):
    return (sorted([morph.normal_forms(str(i))[0].strip(string.punctuation + string.whitespace) for i in word.split(' ')]))

normalize_word_udf = udf(normalize_word, ArrayType(StringType()))

def check_stopwords(word, russian_stopwords = russian_stopwords):
    if word not in russian_stopwords and len(word)>1:
        for letter in word:
            if letter in string.punctuation or letter in string.digits:
                return 0
        return 1
    return 0

check_stopwords_udf = udf(check_stopwords, StringType())

In [4]:
df_tx_grouped_normalize = (spark.read.parquet("df_tx_grouped_filtered")#.limit(3000)
                           .withColumn('new_tx',normalize_word_udf(col('tx')))
                    )

In [5]:
%%time
df_tx_grouped_normalize.write.mode('overwrite').parquet('df_tx_grouped_normalize')

CPU times: total: 656 ms
Wall time: 27min 22s


### По фразам

In [185]:
df_tx_grouped_normalize_agg_str = (
    spark.read.parquet(r"df_tx_grouped_normalize")
    .withColumn("new_tx_str1", f.trim(f.concat_ws(" ",col("new_tx")))).withColumn("new_tx_str", f.expr('''BTRIM(new_tx_str1, '—!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~')'''))
    .where('coalesce(new_tx_str, "") !="" ')
    .groupBy(f.col("rubricName"),f.col("new_tx_str"),f.col("year")) 
    .agg(f.sum("cnt").alias("count"))
    .sort(desc('count'))
    #.show()
)

In [186]:
%%time
df_tx_grouped_normalize_agg_str.write.mode('overwrite').parquet('df_tx_grouped_normalize_agg_str')

CPU times: total: 0 ns
Wall time: 34 s


In [196]:
df_tx_str = spark.read.parquet(r"df_tx_grouped_normalize_agg_str")

df_tx_str.createOrReplaceTempView('df_tx_str')

df_tx_grouped_normalize_final_str = spark.sql('''
with t as
    (
    select
        rubricName
        ,new_tx_str
        ,case when year=2020 then sum(count) else 0 end cnt_2020
        ,case when year=2021 then sum(count) else 0 end cnt_2021
        ,case when year=2022 then sum(count) else 0 end cnt_2022
        ,case when year=2023 then sum(count) else 0 end cnt_2023
    from df_tx_str
    group by 
        rubricName
        ,new_tx_str
        ,year
    )
, t1 as 
    (        
    select
        rubricName
        ,new_tx_str
        ,max(cnt_2020) cnt_2020
        ,max(cnt_2021) cnt_2021
        ,max(cnt_2022) cnt_2022
        ,max(cnt_2023) cnt_2023
    from t
    group by new_tx_str, rubricName
    )
, t2 as 
    (
    select
        'Топ-70 рубрик' as rubricName
        ,new_tx_str
        ,max(cnt_2020) cnt_2020
        ,max(cnt_2021) cnt_2021
        ,max(cnt_2022) cnt_2022
        ,max(cnt_2023) cnt_2023
    from t
    group by 'Топ-70 рубрик', new_tx_str
    )
, t3 as 
    (
    select * from t1
    union all
    select * from t2
    )

select *, cnt_2020+cnt_2021+cnt_2022+cnt_2023 as cnt_total, row_number() over(partition by rubricName order by cnt_2020+cnt_2021+cnt_2022+cnt_2023 desc) rn
from t3
        
''').where('rn < 32')

In [197]:
df_tx_grouped_normalize_final_str.write.mode('overwrite').parquet('df_tx_grouped_normalize_final_str')

In [214]:
pdf_str = spark.read.parquet('df_tx_grouped_normalize_final_str').toPandas()

cols = ['Рубрика', 'Поисковый запрос', 'Количество запросов 2020', 'Количество запросов 2021', 'Количество запросов 2022', 'Количество запросов 2023', 'Количество запросов, итого', 'Рейтинг по запросам, итого']

pdf_str.columns = cols

rubrics = pd.read_excel('files/final_svod_master.xlsx')[['Рубрика']].iloc[:70]

rubrics['Рейтинг рубрики'] = rubrics.index + 1
cols.append('Рейтинг рубрики')

pdf_new_str = rubrics.merge(pdf_str, how='outer')[cols]
pdf_new_str.loc[pdf_new_str['Рубрика'] == 'Топ-70 рубрик', ['Рейтинг рубрики']] = 0
pdf_new_str = pdf_new_str.sort_values('Рейтинг рубрики')
#pdf_new.to_excel('df_tx_grouped_normalize_final_rated.xlsx')

In [215]:
pdf_new_str.to_excel('pdf_new_str_new.xlsx')

### По словам

In [16]:
df_tx_grouped_normalize_agg = (
    spark.read.parquet(r"df_tx_grouped_normalize")
    .withColumn("new_tx_ex", f.explode(f.col("new_tx")))
    .where('coalesce(new_tx_ex, "") !="" ')
    .groupBy(f.col("rubricName"),f.col("new_tx_ex"),f.col("year")) 
    .agg(f.sum("cnt").alias("count"))
    .sort(desc('count'))
    #.show()
)

In [17]:
%%time
df_tx_grouped_normalize_agg.write.mode('overwrite').parquet('df_tx_grouped_normalize_agg')

CPU times: total: 0 ns
Wall time: 6.88 s


In [105]:
df_tx = spark.read.parquet(r"df_tx_grouped_normalize_agg").withColumn('check', check_stopwords_udf(f.col('new_tx_ex'))).where('check=1')

df_tx.createOrReplaceTempView('df_tx')

df_tx_grouped_normalize_final = spark.sql('''
with t as
    (
    select
        rubricName
        ,new_tx_ex
        ,case when year=2020 then sum(count) else 0 end cnt_2020
        ,case when year=2021 then sum(count) else 0 end cnt_2021
        ,case when year=2022 then sum(count) else 0 end cnt_2022
        ,case when year=2023 then sum(count) else 0 end cnt_2023
    from df_tx
    group by 
        rubricName
        ,new_tx_ex
        ,year
    )
, t1 as (        
select
    rubricName
    ,new_tx_ex
    ,max(cnt_2020) cnt_2020
    ,max(cnt_2021) cnt_2021
    ,max(cnt_2022) cnt_2022
    ,max(cnt_2023) cnt_2023
from t
group by new_tx_ex, rubricName
)

select *, cnt_2020+cnt_2021+cnt_2022+cnt_2023 as cnt_total, row_number() over(partition by rubricName order by cnt_2020+cnt_2021+cnt_2022+cnt_2023 desc) rn
from t1
        
''').where('rn < 31')

In [113]:
df_tx_grouped_normalize_final.write.mode('overwrite').parquet('df_tx_grouped_normalize_final')

In [115]:
pdf = spark.read.parquet('df_tx_grouped_normalize_final').toPandas()

In [142]:
cols = ['Рубрика', 'Поисковое слово', 'Количество запросов 2020', 'Количество запросов 2021', 'Количество запросов 2022', 'Количество запросов 2023', 'Количество запросов, итого', 'Рейтинг по запросам, итого']

pdf.columns = cols

rubrics = pd.read_excel('files/final_svod_master.xlsx')[['Рубрика']].iloc[:70]

rubrics['Рейтинг рубрики по запросам, итого'] = rubrics.index + 1

pdf_new = rubrics.merge(pdf, how='outer')

pdf_new.to_excel('df_tx_grouped_normalize_final_rated.xlsx')